In [1]:
import re
import numpy as np

!git clone https://github.com/IlyaSk10/stress_symbol_predict/
%cd stress_symbol_predict
!unzip train_dataset.zip

# чтение словаря
with open("All_Forms_1.txt",'r',encoding='UTF-8') as f:
  file=f.readlines()


ret=[re.findall(r'[^,]+',re.sub(r'\w+#','',i)) for i in file]   



CHARS = ['-', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к',
         'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш',
         'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё']
CHAR_INDICES = dict((c, i+1) for i, c in enumerate(CHARS))


# создание структуры "слово:номер буквы на которую падает ударение"
df={}
list2=[]
words_without_stress=[]
for string in ret:
    for word in string:
        if len(re.findall(r'(.+)\'|(.+)\`',word.strip()))>0:
            #front=re.findall(r'(.+)\'|(.+)\`',word.strip())
            front=re.sub(r'[\`\']','',re.findall(r'(.+)[\`\']',word.strip())[0])
            clean_word=''.join(re.findall(r'[^\'\`]',word.strip()))
            #df.update({clean_word:len(max(*front))})
            df.update({clean_word:len(front)})
    if len(df)>0:
      list2.append(df.copy())
      df.clear()
    else:
      words_without_stress.append(string)
  
print('Слова без разметки',words_without_stress[:20]) # слова без разметки и поэтому не учитываются далее

#np.sum([len(list2[i]) for i in range(len(list2))])
MAXLEN=max(len(word) for k in range(len(list2)) for word in list2[k].keys())
print('Максимальная длина слова с разметкой', MAXLEN, 'символов')

MAXLEN_WITHOUT_STRESS=max(len(word) for k in range(len(ret)) for word in ret[k])
print('Максимальная длина слова без разметки', MAXLEN_WITHOUT_STRESS, 'символов')

Cloning into 'stress_symbol_predict'...
remote: Enumerating objects: 97, done.
remote: Total 97 (delta 0), reused 0 (delta 0), pack-reused 97
Unpacking objects: 100% (97/97), done.
/content/stress_symbol_predict
Archive:  train_dataset.zip
  inflating: All_Forms_1.txt         
Слова без разметки [['\ufeffа\n'], ['абы\n'], ['авансом\n'], ['ага\n'], ['агами\n'], ['агу\n'], ['адажио\n'], ['адски\n'], ['аж\n'], ['азу\n'], ['актёр', 'актёры', 'актёра', 'актёров', 'актёру', 'актёрам', 'актёра', 'актёров', 'актёром', 'актёрами', 'актёре', 'актёрах\n'], ['актёришка', 'актёришки', 'актёришки', 'актёришек', 'актёришке', 'актёришкам', 'актёришку', 'актёришек', 'актёришкой', 'актёришкою', 'актёришками', 'актёришке', 'актёришках\n'], ['актёрка', 'актёрки', 'актёрки', 'актёрок', 'актёрке', 'актёркам', 'актёрку', 'актёрок', 'актёркой', 'актёркою', 'актёрками', 'актёрке', 'актёрках\n'], ['актёрство', 'актёрства', 'актёрства', 'актёрств', 'актёрству', 'актёрствам', 'актёрство', 'актёрства', 'актёрством

In [0]:
# функция преобразования каждого слова в набор чисел
def preparation (list2):
  total=np.sum([len(list2[i]) for i in range(len(list2))])
  A=np.zeros(shape=(total,MAXLEN))
  B=np.zeros(shape=(total,1))
  m=0
  for k in range(len(list2)):
    for i in range(len(list2[k])):
        x = np.zeros((1, MAXLEN)) 
        word=list(list2[k].keys())[i]
        for index, letter in enumerate(word.strip()):
            x[0,index]=CHAR_INDICES[letter] # слово представлено вектором чисел от 0 до 33
        A[m,:]=x
        B[m,:]=list2[k][word]
        m=m+1
  return A,B


In [3]:
import random
list_number_elements=20000
random.shuffle(list2) # перемешиваем данные
X_test,y_test=preparation(list2[:list_number_elements]) # и берем первые 20000 штук элементов списка. Каждый элемент списка это (слово+все словообразования)
X_train,y_train=preparation(list2[list_number_elements:])

from keras.utils.np_utils import to_categorical
y_test_cat = to_categorical(y_test, num_classes=MAXLEN)
y_train_cat = to_categorical(y_train, num_classes=MAXLEN)

Using TensorFlow backend.


In [5]:
#функция конвертирует вектор в слово
def convert(X): 
  word_concat=[]
  for word in X:
    word_concat.append(''.join([key for index,number in enumerate(word) for key,value in CHAR_INDICES.items() if value==number]))
  return word_concat

w=list((key,value) for i in range(len(list2)) for key,value in list2[i].items())
number=np.sum([len(list2[i]) for i in range(len(list2[:list_number_elements]))])

number_of_words=20
word_concat=convert(X_train[:number_of_words])

for i in range(len(y_train_cat[:number_of_words])):
  lab=np.argmax(y_train_cat[i])
  print('Слово после конвертации :',word_concat[i][:lab]+"'"+word_concat[i][lab:],'|','Слово из словаря :',w[number+i][0][:w[number+i][1]]+"'"+w[number+i][0][w[number+i][1]:])

Слово после конвертации : де'ррик | Слово из словаря : де'ррик
Слово после конвертации : де'ррики | Слово из словаря : де'ррики
Слово после конвертации : де'ррика | Слово из словаря : де'ррика
Слово после конвертации : де'рриков | Слово из словаря : де'рриков
Слово после конвертации : де'ррику | Слово из словаря : де'ррику
Слово после конвертации : де'ррикам | Слово из словаря : де'ррикам
Слово после конвертации : де'рриком | Слово из словаря : де'рриком
Слово после конвертации : де'рриками | Слово из словаря : де'рриками
Слово после конвертации : де'ррике | Слово из словаря : де'ррике
Слово после конвертации : де'рриках | Слово из словаря : де'рриках
Слово после конвертации : обстре'лянный | Слово из словаря : обстре'лянный
Слово после конвертации : обстре'лянная | Слово из словаря : обстре'лянная
Слово после конвертации : обстре'лянное | Слово из словаря : обстре'лянное
Слово после конвертации : обстре'лянные | Слово из словаря : обстре'лянные
Слово после конвертации : обстре'лянного

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Embedding,Bidirectional,LSTM

# создание сети
model = Sequential()
vocab_size=len(X_train)
model.add(Embedding(vocab_size, output_dim= len(CHARS), input_length=MAXLEN))
model.add(Bidirectional(LSTM(units=64, input_shape=(MAXLEN, len(CHARS)),dropout = 0.2)))
model.add(Dense(MAXLEN, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# обучение модели
model.fit(X_train, y_train_cat, batch_size = 64,epochs=5,validation_data = (X_test, y_test_cat),verbose = 1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1163184 samples, validate on 364400 samples
Epoch 1/5
1163184/1163184 [==============================] - 3497s 3ms/step - loss: 0.7367 - accuracy: 0.7147 - val_loss: 0.5037 - val_accuracy: 0.8002
Epoch 2/5
1163184/1163184 [==============================] - 3488s 3ms/step - loss: 0.4300 - accuracy: 0.8289 - val_loss: 0.4181 - val_accuracy: 0.8408
Epoch 3/5
1163184/1163184 [==============================] - 3493s 3ms/step - loss: 0.3509 - accuracy: 0.8627 - val_loss: 0.3925 - val_accuracy: 0.8554
Epoch 4/5
1163184/1163184 [==============================] - 3503s 3ms/step - loss: 0.3059 - accuracy: 0.8817 - val_loss: 0.3593 - val_accuracy: 0.8692
Epoch 5/5
1163184/1163184 [==============================] - 3500s 3ms/step - loss: 0.2846 - accuracy: 0.8911 - val_loss: 0.3568 - val_accuracy: 0.8697


In [23]:
from sklearn.metrics import accuracy_score

def accuracy(X_test):
  predicted=model.predict(X_test)
  predicted = np.argmax(predicted, axis=1)
  return accuracy_score(y_test, predicted)

print('Доля правильных ответов','{:.3f}'.format(accuracy(X_test)))

Доля правильных ответов 0.870


In [35]:
number_of_words=100 # возьмем первые 100 слов из test
word_concat_test=convert(X_test[:number_of_words])

correct_predict=[] # список для слов в которых верно проставлено ударение
incorrect_predict=[] # список для слов в которых неверно проставлено ударение

for i in range(len(X_test[:number_of_words])):
  predicted=model.predict(X_test[i].reshape(1,MAXLEN))
  predicted = np.argmax(predicted, axis=1)
  if y_test[i]==predicted:
    correct_predict.append(word_concat_test[i][:int(predicted)]+"'"+word_concat_test[i][int(predicted):])
  else:
    incorrect_predict.append(word_concat_test[i][:int(predicted)]+"'"+word_concat_test[i][int(predicted):])

print([('Правильный ответ :',word_correct) for word_correct in correct_predict])
print([('Неправильный ответ :',word_incorrect) for word_incorrect in incorrect_predict])

[('Правильный ответ :', "рассолоди'ть"), ('Правильный ответ :', "рассоложу'"), ('Правильный ответ :', "рассолоди'м"), ('Правильный ответ :', "рассолоди'те"), ('Правильный ответ :', "рассолоди'т"), ('Правильный ответ :', "рассолодя'"), ('Правильный ответ :', "рассолоди'л"), ('Правильный ответ :', "рассолоди'ла"), ('Правильный ответ :', "рассолоди'ло"), ('Правильный ответ :', "рассолоди'ли"), ('Правильный ответ :', "рассолоди'"), ('Правильный ответ :', "рассолоди'вший"), ('Правильный ответ :', "рассолоди'вшая"), ('Правильный ответ :', "рассолоди'вшее"), ('Правильный ответ :', "рассолоди'вшие"), ('Правильный ответ :', "рассолоди'вшего"), ('Правильный ответ :', "рассолоди'вшей"), ('Правильный ответ :', "рассолоди'вших"), ('Правильный ответ :', "рассолоди'вшему"), ('Правильный ответ :', "рассолоди'вшим"), ('Правильный ответ :', "рассолоди'вшую"), ('Правильный ответ :', "рассолоди'вшею"), ('Правильный ответ :', "рассолоди'вшими"), ('Правильный ответ :', "рассолоди'вшем"), ('Правильный ответ 

In [34]:
path='/content/drive'
# сохранение модели и весов
model_json = model.to_json()
with open(path+"model_3.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights(path+"model_3.h5")
print("Saved model to disk")

Saved model to disk
